In [ ]:
import schiller_lab_tools
import glob
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import HTML

from scipy.cluster.vq import kmeans, whiten

# Data import

In [2]:
wd = schiller_lab_tools.read_hdf5(glob.glob("wd*.h5")[0])
od = schiller_lab_tools.read_hdf5(glob.glob("od*.h5")[0])
md, t = schiller_lab_tools.read_asc(glob.glob("md-cfg*.asc")[0])
md = md.sort_values(by = ['p_id'])
md = md.reset_index()
md = md.drop(['index'], axis = 1)
boxDims = wd.shape

phi = od - wd
phi_fill = schiller_lab_tools.microstructure_analysis.fill(phi)

# Visualization

In [ ]:
x = np.random.randn(32, 30)
y = np.random.randn(32, 30)

ani = schiller_lab_tools.visualization.animate_plot(x, y)
HTML(ani.to_jshtml())

In [ ]:
test = np.random.randn(30, 32, 32)*10

ani = schiller_lab_tools.visualization.animate_colormap(test)
HTML(ani.to_jshtml())

# Microstructure analysis

In [ ]:
k1 = schiller_lab_tools.microstructure_analysis.spherical_first_moment(phi_fill)
D1 = 2*np.pi/k1
D1

In [ ]:
k2 = schiller_lab_tools.microstructure_analysis.second_moment(phi_fill)
D2 = 2*np.pi/np.sqrt(k2)
D2

In [ ]:
S_i = schiller_lab_tools.microstructure_analysis.interface_order(phi_fill)
S_i

In [ ]:
filter_func = lambda x: np.where(x > 0, 1, 0)

phi_bin = schiller_lab_tools.microstructure_analysis.label_regions_hk(phi_fill, filter=filter_func)

fig, axs = plt.subplots(1, 2, figsize = (8, 4))

slc = np.s_[:, boxDims[1]//2, :]

ax = axs[0]
im = ax.imshow(phi_fill[slc])
ax.set_title("raw input")
plt.colorbar(im, ax = ax, orientation = "horizontal", shrink = 0.8)

ax = axs[1]
im = ax.imshow(phi_bin[slc])
ax.set_title("Binarized")
plt.colorbar(im, ax = ax, orientation = "horizontal", shrink = 0.8)

In [ ]:
limit = 2/D1
K, H, A = schiller_lab_tools.microstructure_analysis.curvature(phi_fill, (limit**2, limit))

fig, axs = plt.subplots(1, 2, figsize = (8, 4))
a = 0.6
b = 20

ax = axs[0]
ax.hist(K, alpha = a, edgecolor = "k", bins = b, density = True)
ax.set_title("Gaussian curvature")

ax = axs[1]
ax.hist(H, alpha = a, edgecolor = "k", bins = b, density = True)
ax.set_title("Mean curvature")

In [ ]:
pn = schiller_lab_tools.microstructure_analysis.get_pn(phi_fill, parallel = {'cores':1, 'divs':[1, 1, 1], 'overlap':8})

fig, axs = plt.subplots(2,2, figsize = (6, 6), layout = "constrained")
a = 0.6
b = 20

ax = axs[0, 0]
parameter = 'pore.equivalent_diameter' 
ax.hist(pn[parameter], alpha = a, bins = b, density = True, edgecolor = "k")
ax.set_title(parameter)

ax = axs[0, 1]
parameter = 'pore.inscribed_diameter' 
ax.hist(pn[parameter], alpha = a, bins = b, density = True, edgecolor = "k")
ax.set_title(parameter)

ax = axs[1, 0]
parameter = 'throat.equivalent_diameter' 
ax.hist(pn[parameter], alpha = a, bins = b, density = True, edgecolor = "k")
ax.set_title(parameter)

ax = axs[1, 1]
parameter = 'throat.inscribed_diameter' 
ax.hist(pn[parameter], alpha = a, bins = b, density = True, edgecolor = "k")
ax.set_title(parameter)

In [ ]:
filter_func = lambda x: np.where(x > 0, 1, 0)
tau_s = schiller_lab_tools.microstructure_analysis.taufactor_tortuosity(phi_fill, filter = filter_func)

# Particle Analysis

In [ ]:
positions = md[['x', 'y', 'z']].to_numpy()
orientations = md[['o_x', 'o_y', 'o_z']].to_numpy()
theta, not_in_interface = schiller_lab_tools.particle_analysis.calculate_average_cos_interface_normal(phi_fill, positions, orientations, step_size = 2)

fig, ax = plt.subplots(1, 1, figsize = (4, 4))
a = 0.6
b = 20
bins, counts, patch = ax.hist(theta, alpha = a, bins = b, edgecolor = "k", density = True)
ax.set_title("Angle to interface normal")

In [18]:
orientations = md[['o_z', 'o_y', 'o_x']].to_numpy()
S = schiller_lab_tools.particle_analysis.calculate_nematic_order(orientations, director = [0, 0, 1])

In [ ]:
positions = md[['x', 'y', 'z']].to_numpy()
r, gr = schiller_lab_tools.particle_analysis.calculate_rdf(boxDims, positions)

fig, ax = plt.subplots(1, 1, figsize = (4, 4))
ax.plot(r, gr, ls = "-", marker = "None", markerfacecolor = "None")
ax.set_title("RDF")

In [ ]:
positions = md[['x', 'y', 'z']].to_numpy()
Q4 = schiller_lab_tools.particle_analysis.calculate_ql(boxDims, positions, L = 4)
Q6 = schiller_lab_tools.particle_analysis.calculate_ql(boxDims, positions, L = 6)

test = np.array([Q4, Q6])
test = test.T
# test = whiten(test)

clusters = 1
centers, mean_dist = kmeans(test, k_or_guess = clusters)

fig, ax = plt.subplots(1, 1, figsize = (4, 4))

ax.plot(test[:, 0], test[:, 1], ls = "None", marker = ".", markerfacecolor = "None", color = "tab:blue")
# ax.scatter(test[:, 0], test[:, 1])
for i in range(clusters):
    ax.plot(centers[i,0], centers[i, 1], color = "tab:red", marker = "o", ms = 10)

In [ ]:
positions = md[['x', 'y', 'z']].to_numpy()
w4 = schiller_lab_tools.particle_analysis.calculate_wl(boxDims, positions, L = 4)
w6 = schiller_lab_tools.particle_analysis.calculate_wl(boxDims, positions, L = 6)

test = np.array([w4, w6])
test = test.T
# test = whiten(test)

clusters = 1
centers, mean_dist = kmeans(test, k_or_guess = clusters)

fig, ax = plt.subplots(1, 1, figsize = (4, 4))
ax.plot(test[:, 0], test[:, 1], ls = "None", marker = ".", markerfacecolor = "None")
ax.set_xlabel("w4")
ax.set_ylabel("w6")

for i in range(clusters):
    ax.plot(centers[i,0], centers[i, 1], color = "tab:red", marker = "o", ms = 10)

In [ ]:
plt.plot(Q4, w4, ls = "None", marker = ".", color = "tab:blue", markerfacecolor = "None")
plt.xlabel("Q6")
plt.ylabel("w4")